In [ ]:
import os
import pymysql
import pandas as pd
from pypdf import PdfReader, PdfWriter
import os

DB_CONFIG = {
    'host': '192.168.0.114',
    'port': 3306,
    'user': 'my_pc_user',
    'password': '!Qdhdbrclf56',
    'db': 'greetlounge',
    'charset': 'utf8mb4'
}

conn = pymysql.connect(**DB_CONFIG)
df = pd.read_sql('SELECT * FROM give_works', conn)
mother_path = r'C:\Users\HP\Desktop\Tesla\24q4\지급\지급서류'
doc1_path = os.path.join(mother_path, '세금계산서')
doc2_path = os.path.join(mother_path, '자동차등록증')
doc3_path = os.path.join(mother_path, '지급신청서')
doc4_path = os.path.join(mother_path, '지급신청서_자동완성')

# merged 폴더 경로 생성
merged_path = os.path.join(mother_path, 'merged')
os.makedirs(merged_path, exist_ok=True)

# df의 'RN' 컬럼에 대해 반복
for idx, row in df.iterrows():
    target_filename = str(row['RN'])  # 'RN' 컬럼 값 가져오기
    region_text = str(row['지역'])  # '지역' 컬럼 값 가져오기
    
    # doc4_path에서 파일 찾기
    matching_doc4_file = next(
        (f for f in os.listdir(doc4_path) if target_filename in f and f.lower().endswith('.pdf')), None
    )
    
    # doc1_path에서 파일 찾기
    matching_doc1_file = next(
        (f for f in os.listdir(doc1_path) if target_filename in f and f.lower().endswith('.pdf')), None
    )
    
    # doc2_path 하위 폴더까지 전체 탐색
    matching_doc2_file = None
    for root, dirs, files in os.walk(doc2_path):
        for f in files:
            if target_filename in f and f.lower().endswith('.pdf'):
                matching_doc2_file = os.path.join(root, f)
                break
        if matching_doc2_file:
            break
    
    # doc3_path에서 파일 찾기 (지역 텍스트로 검색)
    matching_doc3_file = None
    if not matching_doc4_file:
        matching_doc3_file = next(
            (f for f in os.listdir(doc3_path) if region_text in f and f.lower().endswith('.pdf')), None
        )
    
    # 파일 존재 여부 확인 (doc4 또는 doc3, doc1, doc2)
    if (matching_doc4_file or matching_doc3_file) and matching_doc1_file and matching_doc2_file:
        writer = PdfWriter()
        
        # doc4가 있으면 doc4 사용, 없으면 doc3 사용
        if matching_doc4_file:
            # doc4 파일 경로 생성 및 첫 페이지 추출
            doc4_full_path = os.path.join(doc4_path, matching_doc4_file)
            doc4_reader = PdfReader(doc4_full_path)
            writer.add_page(doc4_reader.pages[0])  # 1페이지: doc4 첫 페이지
            
            # doc2 파일 첫 페이지 추출 (이미 전체 경로)
            doc2_reader = PdfReader(matching_doc2_file)
            writer.add_page(doc2_reader.pages[0])  # 2페이지: doc2 첫 페이지
            
            # doc1 파일 경로 생성 및 첫 페이지 추출
            doc1_full_path = os.path.join(doc1_path, matching_doc1_file)
            doc1_reader = PdfReader(doc1_full_path)
            writer.add_page(doc1_reader.pages[0])  # 3페이지: doc1 첫 페이지
            
            # doc4의 나머지 페이지들 추가 (4페이지부터)
            for page_num in range(1, len(doc4_reader.pages)):
                writer.add_page(doc4_reader.pages[page_num])
            
            # 병합된 파일 저장 (merged 폴더에)
            output_filename = f"{target_filename}_merged.pdf"
            output_path = os.path.join(merged_path, output_filename)
            
            with open(output_path, 'wb') as output_file:
                writer.write(output_file)
            
            # 원본 파일 삭제
            try:
                os.remove(doc4_full_path)
                os.remove(doc1_full_path)
                os.remove(matching_doc2_file)
                print(f"[{idx}] {target_filename}: 병합 완료 (doc4 사용) - {output_path} (총 {len(writer.pages)}페이지), 원본 파일 삭제 완료")
            except Exception as e:
                print(f"[{idx}] {target_filename}: 병합 완료 (doc4 사용) - {output_path} (총 {len(writer.pages)}페이지), 원본 파일 삭제 실패 - {e}")
        
        else:  # doc3 사용
            # doc3 파일 경로 생성 및 첫 페이지 추출
            doc3_full_path = os.path.join(doc3_path, matching_doc3_file)
            doc3_reader = PdfReader(doc3_full_path)
            writer.add_page(doc3_reader.pages[0])  # 1페이지: doc3 첫 페이지
            
            # doc2 파일 첫 페이지 추출 (이미 전체 경로)
            doc2_reader = PdfReader(matching_doc2_file)
            writer.add_page(doc2_reader.pages[0])  # 2페이지: doc2 첫 페이지
            
            # doc1 파일 경로 생성 및 첫 페이지 추출
            doc1_full_path = os.path.join(doc1_path, matching_doc1_file)
            doc1_reader = PdfReader(doc1_full_path)
            writer.add_page(doc1_reader.pages[0])  # 3페이지: doc1 첫 페이지
            
            # doc3의 나머지 페이지들 추가 (4페이지부터)
            for page_num in range(1, len(doc3_reader.pages)):
                writer.add_page(doc3_reader.pages[page_num])
            
            # 병합된 파일 저장 (merged 폴더에)
            output_filename = f"{target_filename}_merged.pdf"
            output_path = os.path.join(merged_path, output_filename)
            
            with open(output_path, 'wb') as output_file:
                writer.write(output_file)
            
            # 원본 파일 삭제
            try:
                os.remove(doc3_full_path)
                os.remove(doc1_full_path)
                os.remove(matching_doc2_file)
                print(f"[{idx}] {target_filename}: 병합 완료 (doc3 사용) - {output_path} (총 {len(writer.pages)}페이지), 원본 파일 삭제 완료")
            except Exception as e:
                print(f"[{idx}] {target_filename}: 병합 완료 (doc3 사용) - {output_path} (총 {len(writer.pages)}페이지), 원본 파일 삭제 실패 - {e}")
    
    else:
        missing = []
        if not matching_doc4_file and not matching_doc3_file:
            missing.append('doc4/doc3')
        if not matching_doc1_file:
            missing.append('doc1')
        if not matching_doc2_file:
            missing.append('doc2')
        print(f"[{idx}] {target_filename}: 파일 부족 - {', '.join(missing)} 없음")